In [218]:
#import processing
import ds
import ml
import model_saving
import TS
import TA
# Load the necessary packages and modules
#from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import numpy as np
import math as m
import talib
import pickle
from sklearn import svm

from hmmlearn.hmm import GaussianHMM
from sklearn.externals import joblib

In [219]:
filename_train ='/home/octo/Dropbox'+ '/SPY3Apr.csv'

In [220]:
# loading csv file
def get_csv_pd(path):
    #spy_pd=pd.read_csv('C:\\Users\Michal\Dropbox\IB_data\SPY.csv',sep=' ',names=['askPrice','askSize','bidPrice','bidSize'],index_col=0,parse_dates=True)
    #spy_pd=pd.read_csv(path+'\SPY.csv',sep=',',names=['askPrice','askSize','bidPrice','bidSize'],index_col=0,parse_dates=True)
    spy_pd=pd.read_csv(path,sep=',',dtype={'askPrice':np.float32,'askSize':np.float32,
                                           'bidPrice':np.float32,'bidSize':np.float32},index_col=0,parse_dates=True)
    #spy_pd = pd.read_csv(path, usecols=['askPrice','askSize','bidPrice','bidSize'], engine='python', skipfooter=3)
    return spy_pd
def BA(df):
    df.bidPrice=df.loc[:,'bidPrice'].replace(to_replace=0, method='ffill')
    df.bidSize=df.loc[:,'bidSize'].replace(to_replace=0, method='ffill')
    df.askPrice=df.loc[:,'askPrice'].replace(to_replace=0, method='ffill')
    df.askSize=df.loc[:,'askSize'].replace(to_replace=0, method='ffill')
    df=df.dropna()
    return df
def preprocessing_mar(df):
    df=df.dropna()
    # to exclude 0
    #data=data[data['bidPrice']>240]
    #data=data[data['askPrice']>240]
    df=df[df['bidPrice']>df.bidPrice.mean()-df.bidPrice.std()]
    df=df[df['askPrice']>df.askPrice.mean()-df.askPrice.std()]
    df['Open']=(df.askPrice+df.bidPrice)/2
    df['Close']=df.Open.shift(1)
    df['High']=df.askPrice.rolling(10).max()
    df['Low']=df.bidPrice.rolling(10).min()
    df['Volume']=df.askSize+df.bidSize
    vwap=((df.loc[:,'bidPrice']*df.loc[:,'bidSize'])+(df.loc[:,'askPrice']*df.loc[:,'askSize']))/(df.loc[:,'bidSize']+df.loc[:,'askSize'])
    df['spread']=(df.Close-vwap)
    df['Up']=np.where(np.logical_and(df.Close.diff(60)>0.06,df.Close.diff(1)>0.0),1,0)
    df['Dn']=np.where(np.logical_and(df.Close.diff(60)<-0.06,df.Close.diff(1)<0.0),-1,0)
    df['UD']=np.where(np.logical_and(df.Close.diff(60)>0.06,df.Close.diff(1)>0.0),1,
                     np.where(np.logical_and(df.Close.diff(60)<-0.06,df.Close.diff(1)<0.0),-1,0))
    df['sc']=np.where(np.logical_and(df.Close.diff(60)>0.04,df.Close.diff(1)>0.0),1,
                     np.where(np.logical_and(df.Close.diff(60)<-0.04,df.Close.diff(1)<0.0),-1,0))
    df=df.dropna()
    return df

In [221]:
data=get_csv_pd(filename_train)
data=BA(data)
data=preprocessing_mar(data)
data_km=ml.kalman_ma(data)
data['ckm']=data.Close-data_km
data['rsi']=talib.RSI(np.array(data.Close.astype('float64')))
data['atr']=talib.ATR(np.array(data.High.astype('float64')),np.array(data.Low.astype('float64')),np.array(data.Close.astype('float64')),timeperiod=20)
data['mom']=talib.MOM(np.array(data.Close.astype('float64')), timeperiod=5)
data['ret']=np.log(data.Close/data.Close.shift(12))
data=data.dropna()

In [222]:
print(data.groupby('UD').size())

UD
-1      67
 0    2298
 1     122
dtype: int64


In [139]:
print(data.groupby('sc').size())

sc
-1      92
 0    2221
 1     174
dtype: int64


In [223]:
hmm_loaded = joblib.load("markov_hmm.pkl")  
predictions_hmm = hmm_loaded.predict(data[['askPrice', 'askSize', 'bidPrice', 'bidSize','High','Low','spread','ckm','rsi', 'atr','mom']])
data['h_state']=predictions_hmm

In [224]:
np.argmin(np.diag(hmm_loaded.transmat_))

4

In [252]:
filename_rf = 'rf.sav'
rf_loaded = pickle.load(open(filename_rf, 'rb'))
predictions_rf = rf_loaded.predict(data[['askPrice', 'askSize', 'bidPrice', 'bidSize','High','Low','spread','ckm','rsi', 'atr','mom']])

In [253]:
data['prUD']=predictions_rf

In [254]:
data.groupby('prUD').size()

prUD
-1     183
 0    2198
 1     106
dtype: int64

In [255]:
data.groupby('h_state').size()

h_state
0     560
2     904
4    1023
dtype: int64

In [256]:
pd.crosstab(index=data.h_state,columns=data.prUD) 

prUD,-1,0,1
h_state,,,
0,68,492,0
2,25,832,47
4,90,874,59


In [176]:
#data['NAP']=np.where(np.logical_and(data.outlr==-1,np.argmin(np.diag(hmm_loaded.transmat_))==0),-1,1)
data['UD']=np.where(np.logical_and(data.prUD!=0,data.h_state==2),1,-1)

In [177]:
print(data.groupby('UD').size())

UD
-1    1893
 1     594
dtype: int64


In [119]:
pd.crosstab(index=data.NAP,columns=data.prUD) 

prUD,-1,0,1
NAP,,,
-1,497,1813,83
1,24,68,2
